In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [2]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")
train_data.drop(['ID'],axis =1, inplace = True)
test_data.drop(['ID'],axis =1, inplace = True)
out_data = train_data[train_data['class']== -999].index
new_train_data = train_data.drop(out_data)

In [3]:
X_train = new_train_data.drop(['class'], axis=1)
y_train = new_train_data['class']

In [4]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
scaler.transform(X_train)
scaler.fit(test_data)
scaler.transform(test_data)

array([[ 1.52747072e-01,  1.60454967e+00, -6.39206256e-01, ...,
        -1.26987621e+00, -2.65687391e-01, -1.68594206e-01],
       [ 1.96820817e+00,  3.89088651e-01,  2.03708330e-01, ...,
         2.73327118e-01,  1.89399286e-02,  8.33234947e-02],
       [-1.44959748e+00, -1.63592840e+00,  3.14011542e-01, ...,
         7.47206376e-01,  3.37180099e-02,  1.74161616e-01],
       ...,
       [ 1.26463217e+00, -2.59773812e-01,  6.35339861e-01, ...,
         1.25411023e+00,  2.27671536e-01,  1.36708441e-01],
       [-1.18675465e+00, -1.17255651e+00, -8.23475390e-01, ...,
        -1.04407660e+00, -2.23102021e-01, -1.89598514e-01],
       [ 1.00179475e+00,  9.86949330e-01,  1.81359903e+00, ...,
        -1.22618051e+00, -1.80839144e-01, -1.96431244e-03]])

In [5]:
from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()
LE.fit(y_train)
y_train = LE.transform(y_train)

from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train)

In [6]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2) # 주성분을 몇개로 할지 결정
printcipalComponents = pca.fit_transform(X_train)
principalDf = pd.DataFrame(data=printcipalComponents, columns = ['principal component1', 'principal component2'])

In [7]:
pca.explained_variance_ratio_

array([0.71296963, 0.13582114])

In [8]:
sum(pca.explained_variance_ratio_)

0.8487907711036536

In [10]:
X_train['pca1'] = principalDf['principal component1']
X_train['pca2'] = principalDf['principal component2']
X_train.shape

(4789, 10)

In [28]:
input_node = 8
hidden_node = [16, 16]
output_node = 4

model = Sequential()
model.add(Dense(hidden_node[0], input_dim = input_node, kernel_initializer = 'normal', activation = 'relu', name = 'Hidden1'))
model.add(Dense(hidden_node[1], activation = 'relu', name = 'Hidden2'))
model.add(Dense(output_node, activation = 'softmax'))

In [29]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = 'accuracy')

In [30]:
model.fit(X_train, y_train, epochs = 50, batch_size = 10, validation_split = 0.3)

Epoch 1/50
336/336 [==============================] - 1s 2ms/step - loss: 1.0315 - accuracy: 0.5826 - val_loss: 0.8294 - val_accuracy: 0.5957
Epoch 2/50
336/336 [==============================] - 1s 2ms/step - loss: 0.7277 - accuracy: 0.6829 - val_loss: 0.6852 - val_accuracy: 0.7411
Epoch 3/50
336/336 [==============================] - 1s 2ms/step - loss: 0.6210 - accuracy: 0.7488 - val_loss: 0.6201 - val_accuracy: 0.7467
Epoch 4/50
336/336 [==============================] - 1s 2ms/step - loss: 0.5780 - accuracy: 0.7727 - val_loss: 0.6064 - val_accuracy: 0.7397
Epoch 5/50
336/336 [==============================] - 1s 2ms/step - loss: 0.5596 - accuracy: 0.7730 - val_loss: 0.5741 - val_accuracy: 0.7676
Epoch 6/50
336/336 [==============================] - 1s 2ms/step - loss: 0.5477 - accuracy: 0.7786 - val_loss: 0.5619 - val_accuracy: 0.7745
Epoch 7/50
336/336 [==============================] - 1s 2ms/step - loss: 0.5354 - accuracy: 0.7852 - val_loss: 0.5546 - val_accuracy: 0.7738
Epoch 

In [31]:
y_pred = model.predict(test_data)
print(y_pred)

[[9.5013911e-01 4.9603142e-02 1.8649714e-04 7.1187649e-05]
 [4.3480126e-03 5.0748871e-03 9.8588276e-01 4.6943827e-03]
 [5.1337242e-01 4.8600683e-01 9.9659583e-06 6.1074545e-04]
 ...
 [2.1909981e-03 1.7267333e-02 9.3250406e-01 4.8037600e-02]
 [9.9058551e-01 9.4141783e-03 5.6934702e-08 3.6120579e-07]
 [9.7961825e-01 1.8891627e-02 1.4805602e-03 9.6295780e-06]]


In [32]:
predict_list =y_pred.argmax(axis=1)
for i in range(len(predict_list)):
    predict_list[i] += 1
print(predict_list)

[1 3 1 ... 3 1 1]


In [34]:
ID_list = pd.read_csv("test.csv")['ID']
x = [predict_list]
def submission_csv(predict_list):
    for i in range(len(x)):
        submission = pd.DataFrame({
            "ID":ID_list,
            "class":x[i]
        })   
        submission.to_csv('submission_'+'pca3'+'.csv', index= False)
        
submission_csv(x)